# HER2 Classification Training Project Pipeline

This notebook contains the complete pipeline for training a HER2 classifier on whole slide images (WSI).

**Pipeline stages:**
1. WSI discovery and indexing
2. Patch extraction from annotated regions
3. Train/validation dataset generation
4. Phase 1: Patch-level ResNet-50 training

**Requirements:**
- CuCIM or OpenSlide for WSI processing
- PyTorch with CUDA support
- TensorBoard (included with PyTorch)
- Weights & Biases (optional, for experiment tracking): `pip install wandb`

In [ ]:
%load_ext autoreload
%autoreload 2

# Import library
import pandas as pd
import numpy as np
import re
import os
from tqdm import tqdm
from IPython.display import display
from glob import glob
import cv2
from sklearn.model_selection import train_test_split
import openpyxl

# Import dependency
from src.preprocessing.generate_metadata import discover_wsi
from src.preprocessing.xml_to_mask import get_mask
from src.preprocessing.annotation_utils import resolve_annotation_path
from src.preprocessing.extract_patches import process_slide
from src.preprocessing.load_wsi import load_wsi
from src.train.train_phase1 import train_phase1


In [2]:
# Configuration
BASE_DIR = 'data'
SOURCES = [
    'Yale_HER2_cohort',
    'Yale_trastuzumab_response_cohort',
    'TCGA_BRCA_Filtered'
]
OUTPUT_CSV = 'outputs/index/wsi_index.csv'

## Setup Logging

In [ ]:
import logging

log_dir = 'outputs/logs'
os.makedirs(log_dir, exist_ok=True)
log_path = os.path.join(log_dir, '%(asctime)s-output.log')

# Configure logging to file only (no console output in notebook)
logger = logging.getLogger('preprocessing')
if not logger.handlers:
    handler = logging.FileHandler(log_path)
    formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
    handler.setFormatter(formatter)
    logger.addHandler(handler)
    logger.setLevel(logging.INFO)
    # Prevent propagation to avoid duplicate logs
    logger.propagate = False

def log(msg):
    logger.info(msg)

In [4]:
def create_patch_validator(min_std: float = 5.0, min_foreground_ratio: float = 0.02, background_value: int = 245):
    """Return a validator that drops low-contrast or mostly background patches."""
    stats = {'accepted': 0, 'discarded': 0}
    def _validator(patch, meta):
        arr = np.asarray(patch)
        if arr.ndim == 3:
            gray = cv2.cvtColor(arr, cv2.COLOR_RGB2GRAY)
        else:
            gray = arr
        if float(gray.std()) < min_std:
            stats['discarded'] += 1
            return False
        foreground_ratio = float(np.mean(gray < background_value))
        if foreground_ratio < min_foreground_ratio:
            stats['discarded'] += 1
            return False
        stats['accepted'] += 1
        return True
    _validator.stats = stats
    return _validator

In [ ]:
csv_path = discover_wsi(
    base_dir=BASE_DIR, 
    sources=SOURCES, 
    output_path=OUTPUT_CSV
)



# Load and display the results
patch_csv = pd.read_csv(csv_path)

display(patch_csv.head(50))

Processing sources: 100%|██████████| 3/3 [00:01<00:00,  2.70it/s]


,wsi_path,slide_id,slide_name,annotation_name,annotation_path
0,data/Yale_HER2_cohort/SVS/Her2Neg_Case_01.svs,Her2Neg_Case_01,Her2Neg_Case_01.svs,Her2Neg_Case_01.xml,data/Yale_HER2_cohort/Annotations/Her2Neg_Case...
1,data/Yale_HER2_cohort/SVS/Her2Neg_Case_02.svs,Her2Neg_Case_02,Her2Neg_Case_02.svs,Her2Neg_Case_02.xml,data/Yale_HER2_cohort/Annotations/Her2Neg_Case...
2,data/Yale_HER2_cohort/SVS/Her2Neg_Case_03.svs,Her2Neg_Case_03,Her2Neg_Case_03.svs,Her2Neg_Case_03.xml,data/Yale_HER2_cohort/Annotations/Her2Neg_Case...
3,data/Yale_HER2_cohort/SVS/Her2Neg_Case_04.svs,Her2Neg_Case_04,Her2Neg_Case_04.svs,Her2Neg_Case_04.xml,data/Yale_HER2_cohort/Annotations/Her2Neg_Case...
4,data/Yale_HER2_cohort/SVS/Her2Neg_Case_05.svs,Her2Neg_Case_05,Her2Neg_Case_05.svs,Her2Neg_Case_05.xml,data/Yale_HER2_cohort/Annotations/Her2Neg_Case...
5,data/Yale_HER2_cohort/SVS/Her2Neg_Case_06.svs,Her2Neg_Case_06,Her2Neg_Case_06.svs,Her2Neg_Case_06.xml,data/Yale_HER2_cohort/Annotations/Her2Neg_Case...
6,data/Yale_HER2_cohort/SVS/Her2Neg_Case_07.svs,Her2Neg_Case_07,Her2Neg_Case_07.svs,Her2Neg_Case_07.xml,data/Yale_HER2_cohort/Annotations/Her2Neg_Case...
7,data/Yale_HER2_cohort/SVS/Her2Neg_Case_08.svs,Her2Neg_Case_08,Her2Neg_Case_08.svs,Her2Neg_Case_08.xml,data/Yale_HER2_cohort/Annotations/Her2Neg_Case...
8,data/Yale_HER2_cohort/SVS/Her2Neg_Case_09.svs,Her2Neg_Case_09,Her2Neg_Case_09.svs,Her2Neg_Case_09.xml,data/Yale_HER2_cohort/Annotations/Her2Neg_Case...
9,data/Yale_HER2_cohort/SVS/Her2Neg_Case_10.svs,Her2Neg_Case_10,Her2Neg_Case_10.svs,Her2Neg_Case_10.xml,data/Yale_HER2_cohort/Annotations/Her2Neg_Case...


In [ ]:
for row in tqdm(patch_csv.head(200).itertuples(index=False), total=200, desc='Processing slides'):
    process_slide(row, base_dir=BASE_DIR)

Processing slides: 100%|██████████| 200/200 [00:34<00:00,  5.87it/s]


# Generate Train/Val CSV Files
Create CSV files with patch paths and labels for training.

**Label Assignment:**
- **Yale S-* or O-* cases** (e.g., S16-32975, O09-03495): Always Positive (1)
- **TCGA-* cases**: Labels from `case&annotation_counts_clean.xlsx` → `Clinical.HER2.status` column
  - "Positive" → 1
  - "Negative" → 0
- **Other Yale cases**: Labels from directory names (Her2Pos=1, Her2Neg=0)

**Train/val split:** Done by case (not by patch) to prevent data leakage with stratification to maintain label balance.

In [7]:
import os
from glob import glob
from sklearn.model_selection import train_test_split

log("=" * 80)
log("Starting train/val CSV generation")
log("=" * 80)

# Load TCGA label mapping from Excel file
tcga_excel_path = 'data/TCGA_BRCA_Filtered/case&annotation_counts_clean.xlsx'
tcga_labels = {}

if os.path.exists(tcga_excel_path):
    tcga_df = pd.read_excel(tcga_excel_path)
    log(f"Loaded TCGA labels from: {tcga_excel_path}")
    log(f"TCGA cases in Excel: {len(tcga_df)}")
    print(f"Loaded TCGA labels from: {tcga_excel_path}")
    print(f"TCGA cases in Excel: {len(tcga_df)}")
    
    # Create mapping from slide ID to label based on Clinical.HER2.status
    for idx, row in tqdm(tcga_df.iterrows(), total=len(tcga_df), desc='Processing TCGA labels'):
        slide_id = str(row['Slide']).strip()
        her2_status = str(row['Clinical.HER2.status']).strip()
        
        # Map Clinical.HER2.status to label
        if her2_status == 'Positive':
            label = 1  # Positive
        elif her2_status == 'Negative':
            label = 0  # Negative
        else:
            msg = f"Warning: Unknown HER2 status '{her2_status}' for {slide_id}"
            log(msg)
            print(msg)
            continue
        
        tcga_labels[slide_id] = label
    
    pos_count = sum(1 for v in tcga_labels.values() if v == 1)
    neg_count = sum(1 for v in tcga_labels.values() if v == 0)
    log(f"TCGA positive cases: {pos_count}")
    log(f"TCGA negative cases: {neg_count}")
    print(f"TCGA positive cases: {pos_count}")
    print(f"TCGA negative cases: {neg_count}")
else:
    msg = f"Warning: TCGA Excel file not found at {tcga_excel_path}"
    log(msg)
    print(msg)

# Collect all patches with labels
log("Collecting patches from all cases...")
patch_data = []
patches_dir = 'outputs/patches'

# Get all case directories first
case_dirs = [d for d in glob(os.path.join(patches_dir, '*')) if os.path.isdir(d)]
log(f"Found {len(case_dirs)} case directories")

skipped_tcga = []
skipped_unknown = []

for case_dir in tqdm(case_dirs, desc='Collecting patches from cases'):
    case_name = os.path.basename(case_dir)
    
    # Determine label based on case type
    if case_name.startswith('TCGA-'):
        # TCGA cases: look up in Excel file
        # Extract TCGA slide ID (before the UUID)
        # Format: TCGA-A1-A0SP-01Z-00-DX1.UUID
        tcga_id = case_name.split('.')[0] if '.' in case_name else case_name
        
        if tcga_id in tcga_labels:
            label = tcga_labels[tcga_id]
        else:
            msg = f"TCGA case {tcga_id} not found in Excel, skipping"
            skipped_tcga.append(tcga_id)
            tqdm.write(f"Warning: {msg}")
            continue
    
    elif case_name.startswith('S') or case_name.startswith('O'):
        # Yale cases starting with S-* or O-* are positive
        # e.g., S16-32975, O09-03495
        label = 1  # Positive
    
    else:
        # Other Yale cases: determine from directory name
        if 'Her2Pos' in case_name or 'Pos' in case_name:
            label = 1  # Positive
        elif 'Her2Neg' in case_name or 'Neg' in case_name:
            label = 0  # Negative
        else:
            msg = f"Cannot determine label for {case_name}, skipping"
            skipped_unknown.append(case_name)
            tqdm.write(f"Warning: {msg}")
            continue
    
    # Get all PNG files in this directory
    patch_files = glob(os.path.join(case_dir, '*.png'))
    
    for patch_file in patch_files:
        patch_data.append({
            'path': patch_file,
            'label': label,
            'case': case_name
        })

# Log skipped cases
if skipped_tcga:
    log(f"Skipped {len(skipped_tcga)} TCGA cases not found in Excel: {', '.join(skipped_tcga[:5])}{'...' if len(skipped_tcga) > 5 else ''}")
if skipped_unknown:
    log(f"Skipped {len(skipped_unknown)} cases with unknown labels: {', '.join(skipped_unknown[:5])}{'...' if len(skipped_unknown) > 5 else ''}")

log(f"Total patches collected: {len(patch_data)}")
print(f"\nTotal patches found: {len(patch_data)}")

# Create DataFrame
patches_df = pd.DataFrame(patch_data)

# Display label distribution
print("\nLabel distribution:")
print(patches_df['label'].value_counts())
neg_patches = (patches_df['label']==0).sum()
pos_patches = (patches_df['label']==1).sum()
print(f"\nNegative (0): {neg_patches}")
print(f"Positive (1): {pos_patches}")
log(f"Label distribution - Negative: {neg_patches}, Positive: {pos_patches}")

# Display cases per label
cases_by_label = patches_df.groupby('label')['case'].nunique()
neg_cases = cases_by_label.get(0, 0)
pos_cases = cases_by_label.get(1, 0)
print(f"\nNumber of cases:")
print(f"Negative cases: {neg_cases}")
print(f"Positive cases: {pos_cases}")
log(f"Number of cases - Negative: {neg_cases}, Positive: {pos_cases}")

# Split by case to avoid data leakage (patches from same slide stay together)
log("Performing train/val split by case...")
cases = patches_df['case'].unique()
train_cases, val_cases = train_test_split(
    cases, 
    test_size=0.2, 
    random_state=42, 
    stratify=[patches_df[patches_df['case']==c]['label'].iloc[0] for c in cases]
)

train_df = patches_df[patches_df['case'].isin(train_cases)][['path', 'label']]
val_df = patches_df[patches_df['case'].isin(val_cases)][['path', 'label']]

log(f"Train patches: {len(train_df)}, Val patches: {len(val_df)}")
log(f"Train cases: {len(train_cases)}, Val cases: {len(val_cases)}")
print(f"\nTrain patches: {len(train_df)}")
print(f"Val patches: {len(val_df)}")
print(f"Train cases: {len(train_cases)}")
print(f"Val cases: {len(val_cases)}")

# Display split balance
train_neg = (train_df['label']==0).sum()
train_pos = (train_df['label']==1).sum()
val_neg = (val_df['label']==0).sum()
val_pos = (val_df['label']==1).sum()

print(f"\nTrain label distribution:")
print(train_df['label'].value_counts())
print(f"\nVal label distribution:")
print(val_df['label'].value_counts())

log(f"Train split - Negative: {train_neg}, Positive: {train_pos}")
log(f"Val split - Negative: {val_neg}, Positive: {val_pos}")

# Save CSV files
train_csv_path = 'outputs/patches_index_train.csv'
val_csv_path = 'outputs/patches_index_val.csv'

train_df.to_csv(train_csv_path, index=False)
val_df.to_csv(val_csv_path, index=False)

log(f"Saved train CSV to: {train_csv_path}")
log(f"Saved val CSV to: {val_csv_path}")
log("Train/val CSV generation completed successfully")
log("=" * 80)

print(f"\nSaved train CSV to: {train_csv_path}")
print(f"Saved val CSV to: {val_csv_path}")

# Display sample
print("\nSample from train set:")
display(train_df.head())

Loaded TCGA labels from: data/TCGA_BRCA_Filtered/case&annotation_counts_clean.xlsx
TCGA cases in Excel: 182


Processing TCGA labels: 100%|██████████| 182/182 [00:00<00:00, 40900.31it/s]

TCGA positive cases: 92
TCGA negative cases: 90



Total patches found: 629567

Label distribution:
label
0    342205
1    287362
Name: count, dtype: int64

Negative (0): 342205
Positive (1): 287362

Number of cases:
Negative cases: 154
Positive cases: 221

Train patches: 470418
Val patches: 159149
Train cases: 300
Val cases: 75

Train label distribution:
label
0    241301
1    229117
Name: count, dtype: int64

Val label distribution:
label
0    100904
1     58245
Name: count, dtype: int64

Saved train CSV to: outputs/patches_index_train.csv
Saved val CSV to: outputs/patches_index_val.csv

Sample from train set:


,path,label
0,outputs/patches/Her2Neg_Case_01/Her2Neg_Case_0...,0
1,outputs/patches/Her2Neg_Case_01/Her2Neg_Case_0...,0
2,outputs/patches/Her2Neg_Case_01/Her2Neg_Case_0...,0
3,outputs/patches/Her2Neg_Case_01/Her2Neg_Case_0...,0
4,outputs/patches/Her2Neg_Case_01/Her2Neg_Case_0...,0


# Phase 1 — Train ResNet-50 (module)
This trains a patch-level HER2 classifier using ResNet-50.

**Inputs:** Two CSV files with columns `path` and `label` (0 = negative, 1 = positive).

**Outputs:**
- Best model: `outputs/phase1/models/model_phase1.pth`
- Checkpoints:
  - `checkpoint_last.pth`: Latest epoch (auto-saved after each epoch)
  - `checkpoint_epoch_N.pth`: Periodic checkpoints (every 5 epochs)
- Logs/metrics: `outputs/phase1/logs/`
  - `metrics.csv`: Training history
  - `best.json`: Best model metrics
  - `confusion_matrix.csv`: Confusion matrix
  - `classification_report.csv`: Detailed classification metrics
- TensorBoard logs: `outputs/phase1/tensorboard/`
- Weights & Biases logs: Online dashboard (if enabled)

**Checkpoint Resumption:**
- Training automatically resumes from `checkpoint_last.pth` if it exists
- Restores: model weights, optimizer state, epoch number, best metrics, training history
- To start fresh: Delete checkpoint files or set `'resume': False`
- Useful for: interrupted training, hardware failures, experiment continuation

**Memory Optimizations (for 7.6GB GPU):**
- **Input size**: 224×224 (instead of 512×512) → 5.4× less memory per image
- **Batch size**: 8 (instead of 32) → 4× less memory
- **Gradient accumulation**: 4 steps → Simulates effective batch size of 32
- **Mixed precision (FP16)**: ~50% memory reduction with minimal accuracy impact
- **Total memory usage**: ~2-3GB (was ~8GB+ causing OOM)

**Experiment Tracking & Analysis:**

1. **TensorBoard** (Local visualization):
   ```bash
   tensorboard --logdir=outputs/phase1/tensorboard
   ```
   - Real-time training curves
   - Loss, AUC, accuracy metrics
   - Learning rate schedule

2. **Weights & Biases** (Cloud-based analysis):
   - Automatic experiment comparison
   - Hyperparameter tracking
   - Confusion matrix visualization
   - Classification report tables
   - Shareable dashboards
   - Install: `pip install wandb`
   - Login: `wandb login` (one-time setup)

Both logging systems run simultaneously for comprehensive analysis and performance optimization.

## Checkpoint Management (Optional)

Run this cell to manage training checkpoints. Useful for starting fresh or inspecting checkpoint status.

In [8]:
import os
from pathlib import Path

checkpoint_dir = Path('outputs/phase1/models')
checkpoint_last = checkpoint_dir / 'checkpoint_last.pth'

# Check checkpoint status
if checkpoint_last.exists():
    import torch
    try:
        checkpoint = torch.load(checkpoint_last, map_location='cpu')
        print(f"✓ Checkpoint found: {checkpoint_last}")
        print(f"  - Epoch: {checkpoint['epoch']}")
        print(f"  - Best score ({checkpoint.get('best_metrics', {}).get('score_key', 'auc')}): {checkpoint['best_score']:.4f}")
        print(f"  - Training will resume from epoch {checkpoint['epoch'] + 1}")
        
        # Find all periodic checkpoints
        periodic_checkpoints = sorted(checkpoint_dir.glob('checkpoint_epoch_*.pth'))
        if periodic_checkpoints:
            print(f"\n  Periodic checkpoints ({len(periodic_checkpoints)}):")
            for cp in periodic_checkpoints[-3:]:  # Show last 3
                print(f"    - {cp.name}")
    except Exception as e:
        print(f"✗ Checkpoint exists but cannot be loaded: {e}")
        print(f"  Consider deleting corrupted checkpoint: rm {checkpoint_last}")
else:
    print(f"✗ No checkpoint found at {checkpoint_last}")
    print(f"  Training will start from scratch")

# Helper function to clear checkpoints (uncomment to use)
def clear_checkpoints():
    """Remove all checkpoint files to start fresh"""
    checkpoint_files = list(checkpoint_dir.glob('checkpoint_*.pth'))
    for cp in checkpoint_files:
        cp.unlink()
        print(f"Deleted: {cp.name}")
    if checkpoint_files:
        print(f"\nCleared {len(checkpoint_files)} checkpoint(s)")
    else:
        print("No checkpoints to clear")

# Uncomment the line below to clear all checkpoints and start fresh
# clear_checkpoints()

✗ No checkpoint found at outputs/phase1/models/checkpoint_last.pth
  Training will start from scratch


In [11]:
# Set CUDA memory optimization environment variable
import os
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'

# Clear any existing CUDA cache
import torch
if torch.cuda.is_available():
    torch.cuda.empty_cache()
    torch.cuda.reset_peak_memory_stats()
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Total GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")

# Check if wandb is installed
try:
    import wandb
    print(f"Weights & Biases version: {wandb.__version__}")
except ImportError:
    print("Warning: wandb not installed. Install with: pip install wandb")
    print("Will continue with TensorBoard logging only")

# CSV Detail: path (image path), label (0=negative, 1=positive)
# Memory optimizations for 7.6GB GPU:
# - Input size: 512x512 (high resolution for better feature extraction)
# - Reduced batch_size: 32 -> 8 (4x less memory)
# - Gradient accumulation: 4 steps (simulates batch_size=32)
# - Mixed precision (FP16): ~50% memory reduction
# - Aggressive cache clearing: Every 10 batches
# Note: 512x512 uses more memory than 224x224. Monitor GPU usage. Reduce to 224 if OOM occurs.
config_input_size = 256
CFG = {
    'train_csv': 'outputs/patches_index_train.csv',
    'val_csv': 'outputs/patches_index_val.csv',
    'output_dir': 'outputs/phase1',
    'pretrained': True,
    'input_size': config_input_size,  # Full resolution patches
    'batch_size': 8,    # Reduced from 32 to 8
    'accumulation_steps': 4,  # Gradient accumulation to simulate batch_size=32
    'use_amp': True,    # Mixed precision training (FP16) for memory efficiency
    'num_workers': 8,   # Increased to utilize more CPU cores
    'epochs': 10,
    'lr': 1e-4,
    'weight_decay': 1e-5,
    'label_col': 'label',
    'path_col': 'path',
    'save_best_by': 'auc',
    'seed': 42,
    # Experiment tracking
    'use_wandb': True,  # Enable Weights & Biases logging
    'wandb_project': 'her2-classification',
    'wandb_name': f'phase1_resnet50_bs{8}_size{config_input_size}_amp',
    # Checkpoint resumption
    'resume': True,  # Automatically resume from last checkpoint if available
}

log("=" * 80)
log("Starting Phase 1 training (ResNet-50)")
log(f"Configuration: {CFG}")
log(f"Memory optimizations enabled:")
log(f"  - Input size: {CFG['input_size']}x{CFG['input_size']}")
log(f"  - Batch size: {CFG['batch_size']} with {CFG.get('accumulation_steps', 1)} accumulation steps")
log(f"  - Effective batch size: {CFG['batch_size'] * CFG.get('accumulation_steps', 1)}")
log(f"  - Mixed precision (FP16): {CFG.get('use_amp', False)}")
log(f"Experiment tracking:")
log(f"  - TensorBoard: Enabled")
log(f"  - Weights & Biases: {CFG.get('use_wandb', False)}")
log(f"Checkpoint resumption: {CFG.get('resume', False)}")
log("=" * 80)

results = train_phase1(CFG)

log(f"Training completed - Best model: {results['best_model_path']}")
log(f"Best metrics: {results['best_metrics']}")
log(f"TensorBoard logs: {results['tb_dir']}")
if results.get('wandb_url'):
    log(f"Weights & Biases URL: {results['wandb_url']}")
log("=" * 80)

print('\n' + '='*80)
print('TRAINING COMPLETED')
print('='*80)
print(f"Best model: {results['best_model_path']}")
print(f"Logs dir: {results['logs_dir']}")
print(f"TensorBoard dir: {results['tb_dir']}")
print('\nTo view TensorBoard:')
print(f"  tensorboard --logdir={results['tb_dir']}")
if results.get('wandb_url'):
    print('\nWeights & Biases Dashboard:')
    print(f"  {results['wandb_url']}")
print('='*80)

Error in callback <bound method _WandbInit._pre_run_cell_hook of <wandb.sdk.wandb_init._WandbInit object at 0x743e304e9640>> (for pre_run_cell), with arguments args (<ExecutionInfo object at 743ce06a61e0, raw_cell="# Set CUDA memory optimization environment variabl.." transformed_cell="# Set CUDA memory optimization environment variabl.." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell:/media/thanakornbuath/Phone%20SSD/her2-attention-classifier/main.ipynb#X53sZmlsZQ%3D%3D>,),kwargs {}:


AuthenticationError: API key verification failed for host http://localhost:8080. Make sure your API key is valid.

GPU: NVIDIA GeForce RTX 4060 Laptop GPU
Total GPU Memory: 7.62 GB
Weights & Biases version: 0.22.3


socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.


BrokenPipeError: [Errno 32] Broken pipe

socket.send() raised exception.
socket.send() raised exception.
socket.send() raised exception.


Error in callback <bound method _WandbInit._post_run_cell_hook of <wandb.sdk.wandb_init._WandbInit object at 0x743e304e9640>> (for post_run_cell), with arguments args (<ExecutionResult object at 743ce06a7e60, execution_count=11 error_before_exec=None error_in_exec=[Errno 32] Broken pipe info=<ExecutionInfo object at 743ce06a61e0, raw_cell="# Set CUDA memory optimization environment variabl.." transformed_cell="# Set CUDA memory optimization environment variabl.." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell:/media/thanakornbuath/Phone%20SSD/her2-attention-classifier/main.ipynb#X53sZmlsZQ%3D%3D> result=None>,),kwargs {}:


BrokenPipeError: [Errno 32] Broken pipe